# Calculadora de avance

## Datos del proyecto

In [2]:
proyecto = "misanayoc"
mes_en_analisis = 5
anio = 2025

## Librerias necesarias

In [3]:
from dotenv import load_dotenv, find_dotenv
from dotenv import load_dotenv, find_dotenv
from functools import reduce
from google.cloud import firestore
from google.oauth2 import service_account
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from pathlib import Path
from typing import Dict, List, Any
from xlsxwriter.utility import xl_range, xl_rowcol_to_cell
import excel2img
import itertools
import json
import os
import pandas as pd
import pickle
import pprint
import re
import win32com.client as win32
import xlsxwriter

## Funciones utiles

### Fusionar diccionarios

In [4]:
def fusionar_diccionarios(diccionario_de_diccionarios):
    """
    Fusiona diccionarios separados en un diccionario unificado.
    Solo incluye claves donde ambos valores sean diferentes de cero.
    Usa dinámicamente las claves del diccionario de entrada.
    
    Args:
        diccionario_de_diccionarios (dict): Diccionario con estructura:
            {
                "nombre_campo1": {clave: valor, ...},
                "nombre_campo2": {clave: valor, ...}
            }
    
    Returns:
        dict: Diccionario fusionado con estructura {clave: {nombre_campo1: valor, nombre_campo2: valor}}
    """
    # Obtener las claves (nombres de los campos) del diccionario principal
    nombres_campos = list(diccionario_de_diccionarios.keys())
    
    if len(nombres_campos) != 2:
        raise ValueError("El diccionario debe contener exactamente 2 campos")
    
    campo1_nombre = nombres_campos[0]
    campo2_nombre = nombres_campos[1]
    
    campo1_datos = diccionario_de_diccionarios[campo1_nombre]
    campo2_datos = diccionario_de_diccionarios[campo2_nombre]
    
    fusionado = {}
    
    # Obtener todas las claves únicas de ambos diccionarios
    todas_las_claves = set(campo1_datos.keys()) | set(campo2_datos.keys())
    
    for clave in todas_las_claves:
        valor1 = campo1_datos.get(clave, 0)
        valor2 = campo2_datos.get(clave, 0)
        
        # Solo agregar si ambos valores son diferentes de cero
        if valor1 != 0 and valor2 != 0:
            fusionado[clave] = {
                campo1_nombre: valor1,
                campo2_nombre: valor2
            }
    
    return fusionado

### Calculadora de costo total

In [5]:
from typing import TypedDict


class CostoIngenieriaResult(TypedDict):
    costo_directo: float
    gastos_generales: float
    utilidad: float
    subtotal: float
    igv: float
    total: float


def calculadora_costo_total(costo_directo: float, gastos_generales: float) -> CostoIngenieriaResult:
    """
    Calcula el costo total de un proyecto de ingeniería civil en soles peruanos.

    Args:
        costo_directo (float): Costo directo del proyecto

    Returns:
        CostoIngenieriaResult: Diccionario con todos los valores calculados
    """
    # Calcular gastos generales (10% del costo directo)
    gastos_generales = round(costo_directo * gastos_generales, 2)

    # Calcular utilidad (5% del costo directo)
    utilidad = round(costo_directo * 0.05, 2)

    # Calcular subtotal
    subtotal = round(costo_directo + gastos_generales + utilidad, 2)

    # Calcular IGV (18% del subtotal)
    igv = round(subtotal * 0.18, 2)

    # Calcular total
    total = round(subtotal + igv, 2)

    return {
        "costo_directo": round(costo_directo, 2),
        "gastos_generales": gastos_generales,
        "utilidad": utilidad,
        "subtotal": subtotal,
        "igv": igv,
        "total": total,
    }

### Multiplicador de precio unitario actualizado por cargas de trabajo

In [6]:
def calcular_costos_unitarios(
    dict_precios_unitarios_actualizados, cargas_trabajo
):
    """
    Multiplica cada carga de trabajo del contratista por su precio unitario
    correspondiente.

    Si alguna clave en cargas_trabajo tiene carga > 0 y no existe
    en dict_precios_unitarios_actualizados, levanta KeyError.
    Claves con carga = 0 se ignoran (no dan error), y claves en precios que noprecios_uninatrios_expediente
    están en cargas se tratan con carga = 0.

    Args:
        dict_precios_unitarios_actualizados (dict): código → precio unitario.
        cargas_trabajo (dict): código → carga de trabajo.

    Returns:
        dict: código → precio total (precio_unitario * carga).

    Raises:
        KeyError: si alguna clave con carga > 0 en cargas_trabajo
                  no existe en dict_precios_unitarios_actualizados.
    """
    # Detectar faltantes solo para cargas > 0
    faltantes = {
        codigo
        for codigo, carga in cargas_trabajo.items()
        if carga != 0 and codigo not in dict_precios_unitarios_actualizados
    }
    if faltantes:
        raise KeyError(f"Faltan precios unitarios para las claves: {faltantes}")

    # Construir resultado: para cada precio, multiplicar por carga (0 si no hay)
    resultado = {}
    for codigo, precio_unitario in dict_precios_unitarios_actualizados.items():
        carga = cargas_trabajo.get(codigo, 0)
        resultado[codigo] = precio_unitario * carga

    return resultado

### Multiplicador de precio unitario por carga de trabajo v2

In [7]:
def calcular_costos_unitarios_v2(
    dict_precios_unitarios_actualizados, cargas_trabajo
):
    """
    Multiplica cada carga de trabajo del contratista por su precio unitario
    correspondiente.

    Si alguna clave en cargas_trabajo tiene carga > 0 y no existe
    en dict_precios_unitarios_actualizados, levanta KeyError.
    Claves con carga = 0 se ignoran (no dan error), y claves en precios que no
    están en cargas se tratan con carga = 0.

    Args:
        dict_precios_unitarios_actualizados (dict): código → precio unitario.
        cargas_trabajo (dict): código → carga de trabajo.

    Returns:
        dict: código → precio total (precio_unitario * carga).

    Raises:
        KeyError: si alguna clave con carga > 0 en cargas_trabajo
                  no existe en dict_precios_unitarios_actualizados.
    """
    # Detectar faltantes solo para cargas > 0
    faltantes = {
        codigo
        for codigo, carga in cargas_trabajo.items()
        if carga != 0 and codigo not in dict_precios_unitarios_actualizados
    }
    if faltantes:
        raise KeyError(f"Faltan precios unitarios para las claves: {faltantes}")

    # Construir resultado: para cada precio, multiplicar por carga (0 si no hay)
    resultado = {}
    for codigo, precio_unitario in dict_precios_unitarios_actualizados.items():
        carga = cargas_trabajo.get(codigo, 0)
        resultado[codigo] = precio_unitario * carga

    return resultado

### Sumador de diccionarios

In [8]:
from typing import Any, Dict, Union

def sumar_dicts(*dicts: Dict[Any, Any]) -> Dict[Any, Union[int, float]]:
    """
    Suma múltiples diccionarios numéricos.

    Args:
        *dicts: Diccionarios con valores numéricos (int o float).

    Returns:
        Un nuevo diccionario con la suma de valores por clave.

    Raises:
        TypeError: si algún valor no es int ni float o un argumento no es dict.
    """
    result: Dict[Any, Union[int, float]] = {}

    for idx, d in enumerate(dicts, start=1):
        if not isinstance(d, dict):
            raise TypeError(f"El argumento en posición {idx} no es un dict.")
        for key, value in d.items():
            if not isinstance(value, (int, float)):
                raise TypeError(
                    f"El valor para clave '{key}' en el dict {idx} "
                    "no es numérico (int o float)."
                )
            result[key] = result.get(key, 0) + value

    return result

## Carga de datos

### Pickle

In [9]:
# Concatenar la ruta completa al archivo .pkl
ruta_archivo = os.path.join("data",proyecto, f"{proyecto}_cargas_trabajo.pkl")

# Leer el archivo pickle
with open(ruta_archivo, "rb") as f:
    cargas_trabajo_programadas_anualmente = pickle.load(f)


In [10]:
#eliminando el total
cargas_trabajo_programadas_anualmente = cargas_trabajo_programadas_anualmente.iloc[:-1]

cargas_trabajo_programadas_anualmente.tail(15)

,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,TOTAL
codigo,,,,,,,,,,
MR102,116.009174,180.458716,180.458716,180.458716,180.458716,167.568807,180.458716,180.458716,38.669725,1405.00
MR103,0.000000,0.000000,2.890000,0.000000,0.000000,2.890000,0.000000,2.890000,0.000000,8.67
MR104,3.333750,3.333750,3.333750,3.333750,3.333750,3.333750,3.333750,3.333750,0.000000,26.67
MR201,461.538462,769.230769,769.230769,769.230769,769.230769,769.230769,769.230769,769.230769,153.846154,6000.00
MR202,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR203,38.181818,63.636364,50.909091,50.909091,50.909091,50.909091,50.909091,50.909091,12.727273,420.00
MR204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR205,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR206,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


### Firebase

In [11]:
## Datos de # 1. Busca el .env en el directorio actual o en cualquiera de los padres
dotenv_path = find_dotenv()
if not dotenv_path:
    raise FileNotFoundError("No se encontró ningún archivo .env en este directorio ni en sus padres.")
load_dotenv(dotenv_path)

# 2. Define el root del proyecto como la carpeta que contiene el .env
project_root = Path(dotenv_path).parent

# 3. Obtén la ruta relativa de las credenciales desde la variable de entorno
rel_cred_path = os.getenv("FIRESTORE_CREDENTIALS")
if not rel_cred_path:
    raise RuntimeError("No existe la variable FIRESTORE_CREDENTIALS en el .env")

# 4. Construye la ruta absoluta al JSON
cred_path = Path(rel_cred_path)
if not cred_path.is_absolute():
    cred_path = (project_root / cred_path).resolve()

if not cred_path.exists():
    raise FileNotFoundError(f"No existe el archivo de credenciales en: {cred_path}")

# 5. Carga las credenciales y crea el cliente de Firestore
credentials = service_account.Credentials.from_service_account_file(str(cred_path))
client = firestore.Client(credentials=credentials, project=credentials.project_id)

# 6. Prueba que funcione
print("Colecciones disponibles:", [c.id for c in client.collections()])

Colecciones disponibles: ['rutinarios']


In [12]:
db=firestore.Client(credentials=credentials, project=credentials.project_id)

#### Cargas de trabajo

In [13]:
cargas_trabajo_contratista=db.collection("rutinarios").document(proyecto).collection("valorizaciones").document(str(mes_en_analisis)).get().to_dict()

pprint.pprint(cargas_trabajo_contratista)

{'cargas_trabajo_contratista_corregido': {'MR101': 0.57,
                                          'MR102': 180.46,
                                          'MR104': 3.33,
                                          'MR201': 813.08,
                                          'MR203': 63.64,
                                          'MR301': 385.19,
                                          'MR601': 4.15,
                                          'MR701': 2},
 'cargas_trabajo_contratista_inicial': {'MR101': 0.57,
                                        'MR102': 180.46,
                                        'MR104': 3.33,
                                        'MR201': 769.23,
                                        'MR203': 63.64,
                                        'MR301': 385.19,
                                        'MR601': 4.15,
                                        'MR701': 2}}


In [14]:
cargas_trabajo_contratista_corregido=cargas_trabajo_contratista['cargas_trabajo_contratista_corregido']

#### Expediente técnico

In [15]:
doc_proyecto_firebase = db.collection("rutinarios").document(proyecto).get().to_dict()
expediente_tecnico_firebase = doc_proyecto_firebase["expediente"]

pprint.pprint(expediente_tecnico_firebase)

{'cargas_trabajo': {'MR101': 4.4,
                    'MR102': 1405,
                    'MR103': 8.67,
                    'MR104': 26.67,
                    'MR201': 6000,
                    'MR203': 420,
                    'MR301': 3466.67,
                    'MR401': 13.33,
                    'MR601': 29.04,
                    'MR701': 16},
 'codigo_ruta': 'R0405148',
 'coordenadas': {'fin': {'altitud': 3728,
                         'datum': 'WGS84',
                         'hemisferio': 'S',
                         'progresiva': 6600,
                         'x': 189675.12,
                         'y': 8229710.1,
                         'zona': None,
                         'zona_letra': None},
                 'inicio': {'altitud': 3130,
                            'datum': 'WGS84',
                            'hemisferio': 'S',
                            'progresiva': 0,
                            'x': 191509.1,
                            'y': 8225824.5,
        

In [16]:
progresiva_inicio = expediente_tecnico_firebase["coordenadas"][
    "inicio"
]["progresiva"]
progresiva_fin = expediente_tecnico_firebase["coordenadas"][
    "fin"
]["progresiva"]

print(progresiva_inicio)
print(progresiva_fin)

0
6600


In [17]:
gastos_generales_expediente=expediente_tecnico_firebase['gastos_generales']
print(gastos_generales_expediente)

0.15


#### Precios unitarios

In [18]:
precios_unitarios_expediente=expediente_tecnico_firebase["precios_unitarios"]
pprint.pprint(precios_unitarios_expediente)

{'MR101': 265.3,
 'MR102': 10.27,
 'MR103': 21.0,
 'MR104': 17.5,
 'MR201': 0.44,
 'MR202': 78.5,
 'MR203': 5.25,
 'MR204': 0.44,
 'MR205': 105,
 'MR206': 3.6,
 'MR301': 0.14,
 'MR401': 11.42,
 'MR501': 0.53,
 'MR601': 2.1,
 'MR701': 43.75,
 'MR702': 210}


In [19]:
cargas_trabajo_expediente=expediente_tecnico_firebase["cargas_trabajo"]

In [20]:
costos_expediente=fusionar_diccionarios({
    'carga_trabajo':cargas_trabajo_expediente,
    'precio_unitario':precios_unitarios_expediente
})

pprint.pprint(costos_expediente)

{'MR101': {'carga_trabajo': 4.4, 'precio_unitario': 265.3},
 'MR102': {'carga_trabajo': 1405, 'precio_unitario': 10.27},
 'MR103': {'carga_trabajo': 8.67, 'precio_unitario': 21.0},
 'MR104': {'carga_trabajo': 26.67, 'precio_unitario': 17.5},
 'MR201': {'carga_trabajo': 6000, 'precio_unitario': 0.44},
 'MR203': {'carga_trabajo': 420, 'precio_unitario': 5.25},
 'MR301': {'carga_trabajo': 3466.67, 'precio_unitario': 0.14},
 'MR401': {'carga_trabajo': 13.33, 'precio_unitario': 11.42},
 'MR601': {'carga_trabajo': 29.04, 'precio_unitario': 2.1},
 'MR701': {'carga_trabajo': 16, 'precio_unitario': 43.75}}


#### Contrato

In [21]:
contrato = doc_proyecto_firebase["contrato"]
pprint.pprint(contrato)

{'contratista': {'razon_social': 'BUILDIA S.A.C.', 'ruc': ''},
 'denominacion_tramo_convenio': 'EMP. AR-127 - MISANAYOC (KM 6+000)',
 'fecha_inicio': {'anio': 2025, 'dia': 12, 'mes': 4},
 'id_contrato': '002-2025',
 'jefe_mantenimiento': {'apellido': 'Tinta Cáceres',
                        'dni': 0,
                        'nombre': 'Genaro',
                        'titulo': 'Ingeniero'},
 'monto_contrato': 40800.0,
 'numero_cuadrillas': 1,
 'numero_trabajadores': 2,
 'tiempo_ejecucion_dias': 240,
 'tipo_servicio': 'mantenimiento rutinario'}


In [22]:
monto_contrato = contrato["monto_contrato"]
print(monto_contrato)

40800.0


#### Valorizaciones

In [23]:
valorizaciones_ref = db.collection("rutinarios").document(proyecto).collection('valorizaciones')

pprint.pprint(valorizaciones_ref)

In [24]:
docs = valorizaciones_ref.stream()

# Iterar sobre los documentos y acceder a sus datos

valorizaciones={}
for doc in docs:
    print(f'ID del documento: {doc.id}')
    print(f'Datos del documento: {doc.to_dict()}')
    print("-" * 30)

    valorizaciones[doc.id]=doc.to_dict()

pprint.pprint(valorizaciones)

ID del documento: 4
Datos del documento: {'cargas_trabajo_contratista_corregido': {'MR102': 390, 'MR301': 650, 'MR701': 2.6, 'MR101': 0.65}, 'cargas_trabajo_contratista': {'MR102': 390, 'MR301': 650, 'MR701': 2.6, 'MR101': 0.65}, 'avance': {'porcentaje_ejecutado_acumulado_vigente_mes': 0.15267272253391578, 'porcentaje_programado_vigente_mes': 0.07916666666666666, 'porcentaje_programado_acumulado_vigente_mes': 0.07916666666666666, 'porcentaje_ejecutado_vigente_mes': 0.15267272253391578}}
------------------------------
ID del documento: 5
Datos del documento: {'cargas_trabajo_contratista_inicial': {'MR301': 385.19, 'MR203': 63.64, 'MR201': 769.23, 'MR101': 0.57, 'MR102': 180.46, 'MR104': 3.33, 'MR701': 2, 'MR601': 4.15}, 'cargas_trabajo_contratista_corregido': {'MR301': 385.19, 'MR203': 63.64, 'MR201': 813.08, 'MR101': 0.57, 'MR102': 180.46, 'MR104': 3.33, 'MR701': 2, 'MR601': 4.15}}
------------------------------
{'4': {'avance': {'porcentaje_ejecutado_acumulado_vigente_mes': 0.15267272

## Cálculos

### Recalculo de precios unitarios

In [25]:
# Crear DataFrame usando pd.DataFrame.from_dict() con orient='index'
df_costos_expediente = pd.DataFrame.from_dict(
    costos_expediente, orient="index"
)

# Resetear el índice para convertir las claves MR en una columna
df_costos_expediente = df_costos_expediente.reset_index().rename(
    columns={"index": "codigo_MR"}
)

In [26]:
# Agregar columna parcial
df_costos_expediente["parcial"] = (
    df_costos_expediente["precio_unitario"]
    * df_costos_expediente["carga_trabajo"]
)
df_costos_expediente.head()

,codigo_MR,carga_trabajo,precio_unitario,parcial
0,MR301,3466.67,0.14,485.3338
1,MR102,1405.00,10.27,14429.3500
2,MR203,420.00,5.25,2205.0000
3,MR201,6000.00,0.44,2640.0000
4,MR103,8.67,21.00,182.0700


In [27]:
costo_directo = df_costos_expediente["parcial"].sum()
print("costo_directo", costo_directo)

costo_directo 22489.011399999996


In [28]:
costo_total_expediente = calculadora_costo_total(costo_directo,gastos_generales_expediente)
print(costo_total_expediente["total"])

31844.44


In [29]:
df_costos_expediente["precio_unitario_actualizado"] = (
    df_costos_expediente["precio_unitario"]
    * (monto_contrato / costo_total_expediente["total"])
)

df_costos_expediente["parcial_actualizado"] = (
    df_costos_expediente["precio_unitario_actualizado"]
    * df_costos_expediente["carga_trabajo"]
)

# Calcular el total
costo_directo_actualizado = df_costos_expediente["parcial_actualizado"].sum()

print(f"El costo directo es: {costo_directo_actualizado}")

El costo directo es: 28813.559450880595


In [30]:
df_costos_expediente.head(10)

,codigo_MR,carga_trabajo,precio_unitario,parcial,precio_unitario_actualizado,parcial_actualizado
0,MR301,3466.67,0.14,485.3338,0.179372,621.823434
1,MR102,1405.00,10.27,14429.3500,13.158215,18487.292601
2,MR203,420.00,5.25,2205.0000,6.726449,2825.108559
3,MR201,6000.00,0.44,2640.0000,0.563740,3382.442901
4,MR103,8.67,21.00,182.0700,26.905796,233.273250
5,MR701,16.00,43.75,700.0000,56.053741,896.859860
6,MR101,4.40,265.30,1167.3200,339.909887,1495.603503
7,MR104,26.67,17.50,466.7250,22.421496,597.981312
8,MR401,13.33,11.42,152.2286,14.631628,195.039601
9,MR601,29.04,2.10,60.9840,2.690580,78.134431


In [31]:
dict_precios_unitarios_actualizados = dict(
    zip(
        df_costos_expediente["codigo_MR"],
        df_costos_expediente["precio_unitario_actualizado"],
    )
)

pprint.pprint(dict_precios_unitarios_actualizados)

{'MR101': 339.9098869378768,
 'MR102': 13.158215374489236,
 'MR103': 26.905795799831935,
 'MR104': 22.421496499859945,
 'MR201': 0.5637404834250501,
 'MR203': 6.726448949957984,
 'MR301': 0.17937197199887958,
 'MR401': 14.63162800162289,
 'MR601': 2.6905795799831935,
 'MR701': 56.05374124964987}


#### Cálculo de pago de acuerdo a cargas de trabajo

In [32]:
pago_costo_directo_parciales_contratista = calcular_costos_unitarios(
    dict_precios_unitarios_actualizados, cargas_trabajo_expediente
)
pprint.pprint(pago_costo_directo_parciales_contratista)

{'MR101': 1495.603502526658,
 'MR102': 18487.292601157376,
 'MR103': 233.27324958454287,
 'MR104': 597.9813116512647,
 'MR201': 3382.4429005503007,
 'MR203': 2825.1085589823533,
 'MR301': 621.8234341693559,
 'MR401': 195.03960126163312,
 'MR601': 78.13443100271193,
 'MR701': 896.8598599943979}


In [33]:
df_pago_costo_directo_parciales_contratista = pd.DataFrame.from_dict(
    pago_costo_directo_parciales_contratista, orient="index"
)

df_pago_costo_directo_parciales_contratista = (
    df_pago_costo_directo_parciales_contratista.reset_index().rename(
        columns={"index": "codigo_MR"}
    )
)

df_pago_costo_directo_parciales_contratista = (
    df_pago_costo_directo_parciales_contratista.rename(columns={0: "monto_pago"})
)

df_pago_costo_directo_parciales_contratista

,codigo_MR,monto_pago
0,MR301,621.823434
1,MR102,18487.292601
2,MR203,2825.108559
3,MR201,3382.442901
4,MR103,233.273250
5,MR701,896.859860
6,MR101,1495.603503
7,MR104,597.981312
8,MR401,195.039601
9,MR601,78.134431


In [34]:
sumar_valores = lambda d: sum(d.values())

In [35]:
pago_costo_directo_contratista = sumar_valores(pago_costo_directo_parciales_contratista)
print(pago_costo_directo_contratista)

28813.559450880595


In [36]:
pago_costo_total_contratista = calculadora_costo_total(pago_costo_directo_contratista,gastos_generales_expediente)
print(pago_costo_total_contratista["total"])

40800.0


#### Validación

In [37]:
diferencia_costos = abs(
    pago_costo_total_contratista["total"]
    - monto_contrato
)

pprint.pprint(
    {
        "diferencia_costos": diferencia_costos,
        "contrato": monto_contrato,
        "costos_unitarios": pago_costo_total_contratista["total"],
    }
)

if diferencia_costos > 1:
    raise ValueError(
        "La valorizacion de las actividades presentadas por el contratista no es coherente con lo programado"
    )

{'contrato': 40800.0, 'costos_unitarios': 40800.0, 'diferencia_costos': 0.0}


#### Cálculo de cargas de trabajo ejecutadas del contratista

##### Mes en curso

In [38]:
cargas_trabajo_contratista_current_month=valorizaciones[str(mes_en_analisis)]['cargas_trabajo_contratista_corregido']
pprint.pprint(cargas_trabajo_contratista_current_month)

{'MR101': 0.57,
 'MR102': 180.46,
 'MR104': 3.33,
 'MR201': 813.08,
 'MR203': 63.64,
 'MR301': 385.19,
 'MR601': 4.15,
 'MR701': 2}


##### Acumulado

In [39]:
cargas_trabajo_ejecutadas_acumuladas={}
for v in valorizaciones:
    if(int(v)<=mes_en_analisis):
        cargas_trabajo_ejecutadas_acumuladas=sumar_dicts(cargas_trabajo_ejecutadas_acumuladas,valorizaciones[v]['cargas_trabajo_contratista_corregido'])

pprint.pprint(cargas_trabajo_ejecutadas_acumuladas)

{'MR101': 1.22,
 'MR102': 570.46,
 'MR104': 3.33,
 'MR201': 813.08,
 'MR203': 63.64,
 'MR301': 1035.19,
 'MR601': 4.15,
 'MR701': 4.6}


#### Cálculo de cargas de trabajo programadas

##### Mes en curso

In [40]:
cargas_trabajo_programadas_current_month = cargas_trabajo_programadas_anualmente[f"2025-{mes_en_analisis:02d}"].to_dict()
pprint.pprint(cargas_trabajo_programadas_current_month)

{'MR101': 0.5739130434782609,
 'MR102': 180.45871559633028,
 'MR103': 0.0,
 'MR104': 3.33375,
 'MR201': 769.2307692307692,
 'MR202': 0.0,
 'MR203': 63.63636363636364,
 'MR204': 0.0,
 'MR205': 0.0,
 'MR206': 0.0,
 'MR301': 385.18555555555554,
 'MR401': 3.3325,
 'MR501': 0.0,
 'MR601': 0.0,
 'MR701': 2.0,
 'MR702': 0.0}


##### Acumulado

In [41]:
cargas_trabajo_programadas_anualmente.drop(columns=['TOTAL'], inplace=True)

In [42]:
# Asumiendo que tu dataframe se llama 'df'
cargas_trabajo_programadas_anualmente_dict = {}

# Iterar por las columnas del dataframe (excluyendo la columna 'codigo' si existe)
for columna in cargas_trabajo_programadas_anualmente.columns:
    if columna != 'codigo':  # Excluir la columna de códigos
        # Extraer el mes de la columna (formato "2025-MM")
        mes = int(columna.split('-')[1])  # Esto convierte "04" a 4, "12" a 12
        
        # Crear el diccionario interno para este mes
        cargas_trabajo_programadas_anualmente_dict[mes] = {}
        
        # Llenar el diccionario con los códigos y valores
        for codigo in cargas_trabajo_programadas_anualmente.index:  # Asumiendo que 'codigo' es el índice
            cargas_trabajo_programadas_anualmente_dict[mes][codigo] = float(cargas_trabajo_programadas_anualmente.loc[codigo, columna])

pprint.pprint(cargas_trabajo_programadas_anualmente_dict)

{4: {'MR101': 0.19130434782608696,
     'MR102': 116.00917431192661,
     'MR103': 0.0,
     'MR104': 3.33375,
     'MR201': 461.53846153846155,
     'MR202': 0.0,
     'MR203': 38.18181818181818,
     'MR204': 0.0,
     'MR205': 0.0,
     'MR206': 0.0,
     'MR301': 385.18555555555554,
     'MR401': 0.0,
     'MR501': 0.0,
     'MR601': 0.0,
     'MR701': 2.0,
     'MR702': 0.0},
 5: {'MR101': 0.5739130434782609,
     'MR102': 180.45871559633028,
     'MR103': 0.0,
     'MR104': 3.33375,
     'MR201': 769.2307692307692,
     'MR202': 0.0,
     'MR203': 63.63636363636364,
     'MR204': 0.0,
     'MR205': 0.0,
     'MR206': 0.0,
     'MR301': 385.18555555555554,
     'MR401': 3.3325,
     'MR501': 0.0,
     'MR601': 0.0,
     'MR701': 2.0,
     'MR702': 0.0},
 6: {'MR101': 0.5739130434782609,
     'MR102': 180.45871559633028,
     'MR103': 2.8899999999999997,
     'MR104': 3.33375,
     'MR201': 769.2307692307692,
     'MR202': 0.0,
     'MR203': 50.909090909090914,
     'MR204': 0.0,
 

In [43]:
cargas_trabajo_programadas_acumuladas={}

for v in cargas_trabajo_programadas_anualmente_dict:
    if(int(v)<=mes_en_analisis):
        cargas_trabajo_programadas_acumuladas=sumar_dicts(cargas_trabajo_programadas_acumuladas,cargas_trabajo_programadas_anualmente_dict[v])

pprint.pprint(cargas_trabajo_programadas_acumuladas)

{'MR101': 0.7652173913043478,
 'MR102': 296.4678899082569,
 'MR103': 0.0,
 'MR104': 6.6675,
 'MR201': 1230.7692307692307,
 'MR202': 0.0,
 'MR203': 101.81818181818181,
 'MR204': 0.0,
 'MR205': 0.0,
 'MR206': 0.0,
 'MR301': 770.3711111111111,
 'MR401': 3.3325,
 'MR501': 0.0,
 'MR601': 0.0,
 'MR701': 4.0,
 'MR702': 0.0}


#### Avance valorizado ejecutado

##### Mensual

In [44]:
avance_valorizado_ejecutado=calcular_costos_unitarios_v2(dict_precios_unitarios_actualizados=dict_precios_unitarios_actualizados,cargas_trabajo=cargas_trabajo_contratista_current_month)

pprint.pprint(avance_valorizado_ejecutado)

{'MR101': 193.74863555458978,
 'MR102': 2374.5315464803275,
 'MR103': 0.0,
 'MR104': 74.66358334453362,
 'MR201': 458.36611226323976,
 'MR203': 428.0712111753261,
 'MR301': 69.09228989424842,
 'MR401': 0.0,
 'MR601': 11.165905256930253,
 'MR701': 112.10748249929973}


In [45]:
sumar_valores = lambda d: sum(d.values())
costo_directo_avance_valorizado_ejecutado=sumar_valores(avance_valorizado_ejecutado)
print(costo_directo_avance_valorizado_ejecutado)

3721.746766468495


In [46]:
costo_total_avance_valorizado_ejecutado = calculadora_costo_total(costo_directo_avance_valorizado_ejecutado,gastos_generales_expediente)
print(costo_total_avance_valorizado_ejecutado["total"])

5270.0


##### Acumulado

In [47]:
avance_valorizado_ejecutado_acumulado=calcular_costos_unitarios_v2(dict_precios_unitarios_actualizados=dict_precios_unitarios_actualizados,cargas_trabajo=cargas_trabajo_ejecutadas_acumuladas)

pprint.pprint(avance_valorizado_ejecutado_acumulado)

{'MR101': 414.6900620642097,
 'MR102': 7506.23554253113,
 'MR103': 0.0,
 'MR104': 74.66358334453362,
 'MR201': 458.36611226323976,
 'MR203': 428.0712111753261,
 'MR301': 185.68407169352017,
 'MR401': 0.0,
 'MR601': 11.165905256930253,
 'MR701': 257.8472097483894}


In [48]:
sumar_valores = lambda d: sum(d.values())
costo_directo_avance_valorizado_ejecutado_acumulado=sumar_valores(avance_valorizado_ejecutado_acumulado)
print(costo_directo_avance_valorizado_ejecutado_acumulado)

9336.72369807728


In [49]:
costo_total_avance_valorizado_ejecutado_acumulado = calculadora_costo_total(costo_directo_avance_valorizado_ejecutado_acumulado,gastos_generales_expediente)
print(costo_total_avance_valorizado_ejecutado_acumulado["total"])

13220.8


### Avance valorizado programado

#### Mensual

In [50]:
avance_valorizado_programado=calcular_costos_unitarios_v2(dict_precios_unitarios_actualizados=dict_precios_unitarios_actualizados,cargas_trabajo=cargas_trabajo_programadas_current_month)

pprint.pprint(avance_valorizado_ejecutado)

{'MR101': 193.74863555458978,
 'MR102': 2374.5315464803275,
 'MR103': 0.0,
 'MR104': 74.66358334453362,
 'MR201': 458.36611226323976,
 'MR203': 428.0712111753261,
 'MR301': 69.09228989424842,
 'MR401': 0.0,
 'MR601': 11.165905256930253,
 'MR701': 112.10748249929973}


In [51]:
sumar_valores = lambda d: sum(d.values())
costo_directo_avance_valorizado_programado=sumar_valores(avance_valorizado_programado)
print(costo_directo_avance_valorizado_programado)

3735.9931802702217


In [52]:
costo_total_avance_valorizado_programado = calculadora_costo_total(costo_directo_avance_valorizado_programado,gastos_generales_expediente)
print(costo_total_avance_valorizado_programado["total"])

5290.16


#### Acumulado

In [53]:
avance_valorizado_programado_acumulado=calcular_costos_unitarios_v2(dict_precios_unitarios_actualizados=dict_precios_unitarios_actualizados,cargas_trabajo=cargas_trabajo_programadas_acumuladas)

pprint.pprint(avance_valorizado_programado_acumulado)

{'MR101': 260.10495696115794,
 'MR102': 3900.9883470332084,
 'MR103': 0.0,
 'MR104': 149.49532791281618,
 'MR201': 693.8344411385232,
 'MR203': 684.8748021775401,
 'MR301': 138.18298537096797,
 'MR401': 48.75990031540828,
 'MR601': 0.0,
 'MR701': 224.21496499859947}


In [54]:
sumar_valores = lambda d: sum(d.values())
costo_directo_avance_valorizado_programado_acumulado=sumar_valores(avance_valorizado_programado_acumulado)
print(costo_directo_avance_valorizado_programado_acumulado)

6100.455725908221


In [55]:
costo_total_avance_valorizado_programado_acumulado = calculadora_costo_total(costo_directo_avance_valorizado_programado_acumulado,gastos_generales_expediente)
print(costo_total_avance_valorizado_programado_acumulado["total"])

8638.25


### Resultados

In [56]:
porcentaje_avance_ejecutado = (
    costo_total_avance_valorizado_ejecutado["total"] / monto_contrato
)
porcentaje_avance_ejecutado_acumulado = (
    costo_total_avance_valorizado_ejecutado_acumulado["total"] / monto_contrato
)
porcentaje_avance_programado = (
    costo_total_avance_valorizado_programado["total"] / monto_contrato
)
porcentaje_avance_programado_acumulado = (
    costo_total_avance_valorizado_programado_acumulado["total"] / monto_contrato
)

print(
    f"{porcentaje_avance_ejecutado=}\n{porcentaje_avance_ejecutado_acumulado=}\n{porcentaje_avance_programado=}\n{porcentaje_avance_programado_acumulado=}"
)

porcentaje_avance_ejecutado=0.12916666666666668
porcentaje_avance_ejecutado_acumulado=0.3240392156862745
porcentaje_avance_programado=0.1296607843137255
porcentaje_avance_programado_acumulado=0.2117218137254902


In [57]:
data_to_print={
    "porcentaje_avance_ejecutado":porcentaje_avance_ejecutado,
    "porcentaje_avance_ejecutado_acumulado":porcentaje_avance_ejecutado_acumulado,
    "porcentaje_avance_programado":porcentaje_avance_programado,
    "porcentaje_avance_programado_acumulado":porcentaje_avance_programado_acumulado
}

In [58]:
ruta_directorio = os.path.join("output", proyecto, str(mes_en_analisis))
ruta_archivo = os.path.join(ruta_directorio, "cargas_trabajo.json")

# Crear el directorio si no existe
os.makedirs(ruta_directorio, exist_ok=True)
# Guardar el diccionario en un archivo JSON

with open(ruta_archivo, "w", encoding="utf-8") as archivo_json:
    json.dump(data_to_print, archivo_json, ensure_ascii=False, indent=4)